In [1]:
import pandas as pd
from dotenv import load_dotenv

In [2]:
import sys
import os

# Add the path to folder B to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Now you can import the function
from INGESTION.embedding_hg import embed_text

/Users/tanyatonoran/Documents/fish-images-classification-using-llm-captioning/fish-3.11-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("Marine_Fish_Species_Full_Description_test.csv")
df

,Fish Name,Summary Description,Image Links
0,Brownbanded bamboo shark,This small carpet sharkis commonly found insha...,https://object-storage.example.com/brownbanded...
1,Whale shark,The world's largest livingshark and fish speci...,https://object-storage.example.com/whale_shark...
2,Zebra shark,A distinctive carpetshark with fivelongitudina...,https://object-storage.example.com/zebra_shark...
3,Coral catshark,A small bottom-dwellingshark found in coral re...,https://object-storage.example.com/coral_catsh...
4,Blacktip reef shark,A medium-sized reef shark easily identified by...,https://object-storage.example.com/blacktip_re...
...,...,...,...
86,Moorish idol,An iconic reef fishwith distinctiveblack and w...,https://object-storage.example.com/moorish_ido...
87,Little dragonfish,"A small, dragon-like fish withwing-like pector...",https://object-storage.example.com/little_drag...
88,Indian mackerel,"A small, schooling fishimportant in commercial...",https://object-storage.example.com/indian_mack...
89,Trumpetfish,An elongated fish with a tube-like snout used ...,https://object-storage.example.com/trumpetfish...


In [6]:
embeddings = embed_text(df['Summary Description'])

In [37]:
for i in range(10):
    print(embeddings['predictions'][0]['values'][i][0])

This small carpet sharkis commonly found inshallow coastal watersand coral reefsthroughout the IndoWest Pacific. Youngindividuals have distinctbrown banding thatfades to solid brown asthey mature. They arenocturnal feeders thatcan survive up to 12hours out of water.
The world's largest livingshark and fish species,reaching lengths up to18.8 meters. Thesegentle giants are filterfeeders with distinctivecheckerboard patternsof spots and stripes.They are highlymigratory and canundertake journeys of upto 13,000km.
A distinctive carpetshark with fivelongitudinal ridges and atail nearly half its totalbody length. Juvenilesdisplay zebra-like stripeswhile adults haveleopard-like spots on apale background. Theyare nocturnal huntersfeeding on molluscs,crustaceans, and smallfish.
A small bottom-dwellingshark found in coral reefenvironments. Featuresmarbled patterns on itsbody and is primarilynocturnal, feeding on smallfish and invertebrates.
A medium-sized reef shark easily identified by the prom

In [7]:
embeddings

array([[-3.84656303e-02,  2.82822885e-02, -2.95761302e-02, ...,
        -3.50991562e-02,  5.35659641e-02,  8.99125729e-03],
       [ 3.36583704e-03,  1.24384705e-02, -5.35417907e-02, ...,
         4.74882638e-03, -1.53158037e-02, -2.60087382e-02],
       [-1.91407539e-02,  1.96394771e-02,  2.11337511e-03, ...,
        -5.90370856e-02,  2.87614372e-02,  2.38203760e-02],
       ...,
       [-7.43244216e-02, -1.04054995e-02, -5.07658697e-04, ...,
         4.49527652e-05,  2.72804759e-02,  7.59436516e-03],
       [-4.55915146e-02,  1.38439927e-02, -1.31898362e-03, ...,
        -2.31498070e-02,  1.43344346e-02,  3.98925506e-03],
       [-9.00428221e-02, -1.18127177e-02, -1.23859448e-02, ...,
        -1.42871169e-02, -3.88575234e-02, -2.56530326e-02]],
      shape=(91, 1024), dtype=float32)

In [9]:
df['embedding'] = list(embeddings)

array([-0.03846563,  0.02828229, -0.02957613, ..., -0.03509916,
        0.05356596,  0.00899126], shape=(1024,), dtype=float32)

# Manage ES DB

In [ ]:
from elasticsearch import Elasticsearch

class ElasticsearchManager:
    def __init__(self, es_endpoint, es_username, es_password):
        self.es = Elasticsearch(
    [es_endpoint],
    http_auth=(es_username, es_password),
    verify_certs=False
)
    
    def create_index(self, index_name, mapping=None):
        try:
            if mapping:
                response = self.es.indices.create(index=index_name, body=mapping)
            else:
                response = self.es.indices.create(index=index_name)
            print(f"✓ Index '{index_name}' created successfully")
            return response
        except Exception as e:
            print(f"✗ Error creating index: {e}")
    
    def delete_index(self, index_name):
        try:
            response = self.es.indices.delete(index=index_name)
            print(f"✓ Index '{index_name}' deleted successfully")
            return response
        except Exception as e:
            print(f"✗ Error deleting index: {e}")
    
    def list_indices(self, creator="user"):
        """
        Args:
            creator (str):
                - "user": indices not starting with '.'
                - "system": indices starting with '.'
                - "all": all indices categorized
        """
        try:
            # Fixed: Use keyword argument
            user_index = []
            system_index = []
            indices = self.es.indices.get_alias()
            for index_name in indices.keys():
                count = self.get_document_count(index_name, silent=True)
                if not(index_name.startswith('.')):
                    # print(f"  - {index_name} ({count} documents)")
                    user_index.append(index_name)
                else:
                    system_index.append(index_name)
            print(f"User Indices: {user_index}")
            if creator == "user":
                return user_index
            elif creator == "system":
                return system_index
            elif creator == "all":
                return indices.keys()
        except Exception as e:
            print(f"✗ Error listing indices: {e}")
    
    def get_document_count(self, index_name, silent=False):
        try:
            response = self.es.count(index=index_name)
            count = response['count']
            if not silent:
                print(f"📊 Index '{index_name}' contains {count} documents")
            return count
        except Exception as e:
            if not silent:
                print(f"✗ Error getting document count: {e}")
            return 0

# Usage example
# if __name__ == "__main__":
#     # Initialize manager
#     es_manager = ElasticsearchManager()
    
#     # Create index
#     es_manager.create_index("test_index")
    
#     # List all indices
#     es_manager.list_indices()
    
#     # Check document count
#     es_manager.get_document_count("test_index")
    
#     # Delete index
#     es_manager.delete_index("test_index")

In [24]:
from elasticsearch import Elasticsearch

es_endpoint = os.environ["es_endpoint"]
es_cert_path = os.environ["es_cert_path"]
es_username = os.environ["es_username"]
es_password = os.environ["es_password"]

es_manager = ElasticsearchManager(es_endpoint, es_username, es_password)

/Users/tanyatonoran/Documents/fish-images-classification-using-llm-captioning/fish-3.11-env/lib/python3.11/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://39e00f04-ed4a-4183-992b-4f9039f68877.bkvfu0nd0m8k95k94ujg.databases.appdomain.cloud:30428' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/var/folders/m9/g7cnndw57zg4cpnn660rb4g00000gn/T/ipykernel_16364/593789398.py:5: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  self.es = Elasticsearch(


In [18]:
es_manager.create_index('aa')

/Users/tanyatonoran/Documents/fish-images-classification-using-llm-captioning/fish-3.11-env/lib/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host '39e00f04-ed4a-4183-992b-4f9039f68877.bkvfu0nd0m8k95k94ujg.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


✓ Index 'aa' created successfully


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'aa'})

In [25]:
# # Initialize manager
# es_manager = ElasticsearchManager()

# # Create index
# es_manager.create_index("test_index")

# List all indices
es_manager.list_indices()

# # Check document count
# es_manager.get_document_count("thai_fish_descriptions")

# # Delete index
# es_manager.delete_index("aa")

/Users/tanyatonoran/Documents/fish-images-classification-using-llm-captioning/fish-3.11-env/lib/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host '39e00f04-ed4a-4183-992b-4f9039f68877.bkvfu0nd0m8k95k94ujg.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


📋 All indices:


/Users/tanyatonoran/Documents/fish-images-classification-using-llm-captioning/fish-3.11-env/lib/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host '39e00f04-ed4a-4183-992b-4f9039f68877.bkvfu0nd0m8k95k94ujg.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/tanyatonoran/Documents/fish-images-classification-using-llm-captioning/fish-3.11-env/lib/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host '39e00f04-ed4a-4183-992b-4f9039f68877.bkvfu0nd0m8k95k94ujg.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/tanyatonoran/Documents/fish-images-classification-using-llm-captioning/fi

User Indices: ['aa', 'thai_fish_descriptions', 'product-index']


['aa', 'thai_fish_descriptions', 'product-index']

In [26]:
from INGESTION.elasticsearch_manager import ElasticsearchManager

In [27]:
def get_index_info(self, index_name):
    try:
        if not self.es.indices.exists(index=index_name):
            print(f"❌ Index '{index_name}' not found")
            return None
        
        # Get mapping and count
        mapping = self.es.indices.get_mapping(index=index_name)[index_name]['mappings']
        count = self.es.count(index=index_name)['count']
        
        # Get sample document
        sample = self.es.search(index=index_name, body={"size": 1})
        sample_doc = sample['hits']['hits'][0]['_source'] if sample['hits']['hits'] else {}
        
        print(f"📊 {index_name}: {count} rows")
        print("Columns:")
        for field, value in sample_doc.items():
            field_type = type(value).__name__
            print(f"  {field}: {field_type}")
        
        return {'rows': count, 'columns': list(sample_doc.keys()), 'sample': sample_doc}
        
    except Exception as e:
        print(f"✗ Info error: {e}")

def get_by_fish_id(self, index_name, fish_id):
    try:
        response = self.es.search(
            index=index_name,
            body={"query": {"term": {"fishID": fish_id}}, "size": 1}
        )
        
        if response['hits']['hits']:
            doc = response['hits']['hits'][0]['_source']
            print(f"🐟 Fish {fish_id}:")
            for field, value in doc.items():
                print(f"  {field}: {value}")
            return doc
        else:
            print(f"❌ Fish {fish_id} not found")
            return None
            
    except Exception as e:
        print(f"✗ Retrieve error: {e}")

def get_multiple_fish(self, index_name, fish_ids):
    try:
        response = self.es.search(
            index=index_name,
            body={"query": {"terms": {"fishID": fish_ids}}, "size": len(fish_ids)}
        )
        
        results = []
        for hit in response['hits']['hits']:
            results.append(hit['_source'])
        
        print(f"🐟 Found {len(results)}/{len(fish_ids)} fish")
        return results
        
    except Exception as e:
        print(f"✗ Retrieve error: {e}")

In [30]:
get_index_info(es_manager, 'fish_descriptions')

/Users/tanyatonoran/Documents/fish-images-classification-using-llm-captioning/fish-3.11-env/lib/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host '39e00f04-ed4a-4183-992b-4f9039f68877.bkvfu0nd0m8k95k94ujg.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/tanyatonoran/Documents/fish-images-classification-using-llm-captioning/fish-3.11-env/lib/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host '39e00f04-ed4a-4183-992b-4f9039f68877.bkvfu0nd0m8k95k94ujg.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/tanyatonoran/Documents/fish-images-classification-using-llm-captioning/fi

📊 fish_descriptions: 91 rows
Columns:
  fish_name: str
  general_description: str
  image_links: str
  embedding: list


{'rows': 91,
 'columns': ['fish_name', 'general_description', 'image_links', 'embedding'],
 'sample': {'fish_name': 'Brownbanded bamboo shark',
  'general_description': 'This small carpet sharkis commonly found inshallow coastal watersand coral reefsthroughout the Indo\x02West Pacific. Youngindividuals have distinctbrown banding thatfades to solid brown asthey mature. They arenocturnal feeders thatcan survive up to 12hours out of water.',
  'image_links': 'https://object-storage.example.com/brownbanded_bamboo_shark_1.jpg, https://object-storage.example.com/brownbanded_bamboo_shark_2.jpg',
  'embedding': [-0.03846552595496178,
   0.028282254934310913,
   -0.02957620471715927,
   -0.03304575756192207,
   -0.05324656888842583,
   -3.35857730533462e-05,
   -0.05017129331827164,
   -0.04958188906311989,
   -0.04254204407334328,
   0.05646589770913124,
   0.05120445042848587,
   0.03283994644880295,
   -0.016734395176172256,
   0.07672102749347687,
   0.08306551724672318,
   -0.0005485315923